In [1]:
import pandas as pd
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

def load_and_prepare_csv(path):
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()
    df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.sort_values("datetime", inplace=True)
    return df

def calculate_arbitrage_spread_pct(df):
    df["second"] = df["datetime"].dt.floor("s")

    grouped = df.groupby("second", group_keys=False)
    arbitrage_df = grouped.apply(lambda g: pd.Series({
        "spread_pct": (g["bid1_price"].max() - g["ask1_price"].min()) / g["ask1_price"].min() * 100,
        "best_buy_ex": g.loc[g["ask1_price"].idxmin(), "exchange"],
        "best_sell_ex": g.loc[g["bid1_price"].idxmax(), "exchange"]
    })).reset_index()

    return arbitrage_df

def plot_orderbook_with_arbitrage_pct(df, save_dir):
    df["second"] = df["datetime"].dt.floor("s")
    arbitrage_df = calculate_arbitrage_spread_pct(df)

    exchanges = df['exchange'].unique()
    symbol = df["symbol"][0]
    color_pool = px.colors.qualitative.Plotly
    color_map = {exch: color_pool[i % len(color_pool)] for i, exch in enumerate(exchanges)}

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.1,
                        subplot_titles=(f"{symbol} Bid1 / Ask1 Price Trends", "Arbitrage Spread (%)"))

    # 📈 Row 1：各交易所 Bid / Ask 折线图
    for exchange in exchanges:
        sub_df = df[df['exchange'] == exchange]
        color = color_map[exchange]

        fig.add_trace(go.Scatter(
            x=sub_df["datetime"],
            y=sub_df["bid1_price"],
            mode="lines",
            name=f"{exchange} Bid",
            line=dict(color=color, width=1, dash="dash"),
            opacity=0.7,
            hovertemplate=f"{exchange} Bid<br>%{{x}}<br>%{{y}}<extra></extra>"
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=sub_df["datetime"],
            y=sub_df["ask1_price"],
            mode="lines",
            name=f"{exchange} Ask",
            line=dict(color=color, width=1, dash="solid"),
            opacity=0.7,
            hovertemplate=f"{exchange} Ask<br>%{{x}}<br>%{{y}}<extra></extra>"
        ), row=1, col=1)

    # 🔶 Row 2：套利差价百分比 + 最佳交易所展示
    fig.add_trace(go.Scatter(
        x=arbitrage_df["second"],
        y=arbitrage_df["spread_pct"],
        mode="lines+markers",
        name="Arbitrage (%)",
        line=dict(color="orange", width=2),
        opacity=0.9,
        hovertemplate=(
            "Time: %{x}<br>"
            "Spread: %{y:.4f}%<br>"
            "Best Buy Exchange: %{customdata[0]}<br>"
            "Best Sell Exchange: %{customdata[1]}<extra></extra>"
        ),
        customdata=arbitrage_df[["best_buy_ex", "best_sell_ex"]].values
    ), row=2, col=1)

    fig.update_layout(
        title=f"{symbol} Across Exchanges – Price & Arbitrage %",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis2_title="Time",
        yaxis2_title="Spread (%)",
        hovermode="x unified",
        template="plotly_white",
        legend_title="Type",
        height=700
    )
    symbol = symbol.replace("/", "_").replace(":", "_")
    # fig.show()
    os.makedirs(save_dir, exist_ok=True)
    html_path = os.path.join(save_dir, f"{symbol}.html")

    fig.write_html(html_path, auto_open=True)  # 👈 自动打开图表
    print(f"✅ 可交互图已保存并显示：{html_path}")

# 🌀 扫描 CSV 文件夹并处理
# csv_dir = "../csv_orderbooks_symbol"
# for fname in os.listdir(csv_dir):
#     if fname.endswith(".csv"):
#         symbol = os.path.splitext(fname)[0]
#         try:
#             df = load_and_prepare_csv(os.path.join(csv_dir, fname))
#             plot_orderbook_with_arbitrage_pct(df, symbol)
#         except Exception as e:
#             print(f"⚠️ Error processing {fname}: {e}")
#         break  # 如需批量处理可删除 break


In [ ]:
df = load_and_prepare_csv(os.path.join("../csv_orderbooks_two2/", 'inner_orderbook_ascendex_ADA_USDT_USDT.csv'))
plot_orderbook_with_arbitrage_pct(df, "out_html")

In [10]:
df["symbol"][0]

'ADA/USDT:USDT'